# Assignmnet 4

Generally, a parameter selection procedure might be necessary to evaluate Probability of
Detection versus Probability of False Alarm (i.e., Pd versus Pf) in order to select a classifier
model and/or select a value for a hyperparameter for a classifier.
                                                
In this assignment we will produce an ROC plot presenting operating points of various
classifiers and their varying hyperparameters so that we can make a justifiable operating
classifier/parameter selection for the following problem.

The classification of fake news or misinformation is a very important task today. Download the
fake news dataset (https://www.kaggle.com/clmentbisaillon/fake-and-real-news-dataset),
Fake.csv and True.csv files. Load the datasets into your model development framework and
examine the features to confirm that they are text in title and text columns. Set fake as 1
and true as 0. Concatenate the datasets together to produce one dataset of around 44,880
rows. Apply necessary pre-processing to extract the title column with Tf-Idf. (This assigns
numerical values to terms based on their frequency in a given document and throughout a
given collection of documents.) Use around 50 features. Make sure to include a sanity check in
the pipeline and perhaps run your favorite baseline classifier first.

```
df_true['class'] = 0; df_fake['class'] = 1
df = pd.concat([df_fake, df_true])
X = TfidfVectorizer(stop_words='english',
max_features=40).fit_transform(df['title'])
```

## 1. [70 pts]

By using three classifiers—decision tree, random forest, and neural network—and
at least 2 different hyperparameter settings for each, generate operating points and plot
them on a ROC. In particular, plot mean TPR and mean FPR, where the means are taken
from the multiple runs of cross-validations. Do not hesitate to use/modify the ROC plot code
in the module notebook if necessary. In case you do not see enough variety in Pd-Pf you
might need to work on the classifiers set and/or hyperparameters. And do not hesitate to try
hundreds, if necessary, since the ROC is just a natural scatter plot.
(Some recommended parameters and ranges: depth [3-12], number of features [3-20],
number of estimators [20-100], layer size [1-10], learning rate; and total of 10-20 Ops.)

In [2]:
# Load the datasets 
import pandas as pd

df_true = pd.read_csv('datasets/True.csv')
df_fake = pd.read_csv('datasets/Fake.csv')

In [3]:
# Inspect the dataset
print('\n\n\nTrue dataset head: \n',df_true.head(n=2))
print('\n\n\nFake dataset head: \n',df_fake.head(n=2))
print(f'\n\n\n{df_true.columns}')
print(f'\n\n\n{df_fake.columns}')

df_true['class'] = 0; df_fake['class'] = 1
df = pd.concat([df_fake, df_true])
print(f'\n\n\nThe columns of the final dataset are: {df.columns}')
print(f'dataframe has {len(df)} samples.')




True dataset head: 
                                                title  \
0  As U.S. budget fight looms, Republicans flip t...   
1  U.S. military to accept transgender recruits o...   

                                                text       subject  \
0  WASHINGTON (Reuters) - The head of a conservat...  politicsNews   
1  WASHINGTON (Reuters) - Transgender people will...  politicsNews   

                 date  
0  December 31, 2017   
1  December 29, 2017   



Fake dataset head: 
                                                title  \
0   Donald Trump Sends Out Embarrassing New Year’...   
1   Drunk Bragging Trump Staffer Started Russian ...   

                                                text subject  \
0  Donald Trump just couldn t wish all Americans ...    News   
1  House Intelligence Committee Chairman Devin Nu...    News   

                date  
0  December 31, 2017  
1  December 31, 2017  



Index(['title', 'text', 'subject', 'date'], dtype='object')



Ind

### Split the data into training and testing, and try a 

Per the assignment prompt do a sanity-- in this case check that we have the correct number of samples, approximately 44,000

In [5]:
# Transform the titles into a vector
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split

MAX_FEAUTRES = 50
X = TfidfVectorizer(stop_words='english', max_features=MAX_FEAUTRES).fit_transform(df['title'])
y = df['class']
print(X.shape, y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

(44898, 50) (44898,)


### Test GridSearch optimal paramters across several classifiers

In [11]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV, HalvingRandomSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
import numpy as np


clfs = {
    'decision_tree': DecisionTreeClassifier(random_state=42)
}
param_dict = {
    'decision_tree': {
                        'criterion':['gini', 'entropy', 'log_loss'],
                        'max_features': ['sqrt', 'log2', None],
                        'splitter': ['best', 'random'],
                        'max_depth': [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37],
                        'ccp_alpha': np.linspace(0.0, 10.0, 100),
                        'random_state': [None, 42],
                        'min_samples_leaf': [1, 2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37],
                     },
}


grid_search = GridSearchCV(clfs['decision_tree'], param_dict['decision_tree'], cv=5, scoring='accuracy', n_jobs=-1, verbose=1)

grid_search.fit(X_train, y_train)

# Print the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Accuracy:", grid_search.best_score_)

# Evaluate on test data
best_model = grid_search.best_estimator_
test_accuracy = best_model.score(X_test, y_test)
print("Test Accuracy:", test_accuracy)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Best Parameters: {'criterion': 'gini', 'max_depth': 31, 'max_features': None, 'splitter': 'best'}
Best Accuracy: 0.814633203489928
Test Accuracy: 0.8191536748329621


In [9]:
half_search = HalvingRandomSearchCV(clfs['decision_tree'], param_dict['decision_tree'], scoring='accuracy', n_jobs=-1, verbose=1)
half_search.fit(X_train, y_train)
print("Best Parameters:", half_search.best_params_)
print("Best Accuracy:", half_search.best_score_)
# Evaluate on test data
best_model = half_search.best_estimator_
test_accuracy = best_model.score(X_test, y_test)
print("Test Accuracy:", test_accuracy)

n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 7
min_resources_: 20
max_resources_: 35918
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 216
n_resources: 20
Fitting 5 folds for each of 216 candidates, totalling 1080 fits


/home/ace/work/JHU-AppliedML-EN705601-SP25/venv/lib/python3.12/site-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 216 is smaller than n_iter=1795. Running 216 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


----------
iter: 1
n_candidates: 72
n_resources: 60
Fitting 5 folds for each of 72 candidates, totalling 360 fits
----------
iter: 2
n_candidates: 24
n_resources: 180
Fitting 5 folds for each of 24 candidates, totalling 120 fits
----------
iter: 3
n_candidates: 8
n_resources: 540
Fitting 5 folds for each of 8 candidates, totalling 40 fits
----------
iter: 4
n_candidates: 3
n_resources: 1620
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'splitter': 'best', 'max_features': None, 'max_depth': 31, 'criterion': 'entropy'}
Best Accuracy: 0.7874135229140389
Test Accuracy: 0.8173719376391982


In [14]:
bests = {}
for k in clfs.keys():
    gs = GridSearchCV(clfs[k], param_dict[k], cv=5, scoring='accuracy', n_jobs=-1, verbose=0)
    hs = HalvingRandomSearchCV(clfs[k], param_dict[k], cv=5, scoring='accuracy', n_jobs=-1, verbose=0)
    gs.fit(X_train, y_train)
    hs.fit(X_train, y_train)
    bests[k+'__GridSearch'] = { 
                'best_model': gs.best_estimator_,
                'best_accuracy': gs.best_estimator_.score(X_test, y_test),
                'best_params': gs.best_params_,
                'best_score': gs.best_score_
            }
    bests[k+'__HalvingSearch'] = { 
                'best_model': hs.best_estimator_,
                'best_accuracy': hs.best_estimator_.score(X_test, y_test),
                'best_params': hs.best_params_,
                'best_score': hs.best_score_
            }

/home/ace/work/JHU-AppliedML-EN705601-SP25/venv/lib/python3.12/site-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 216 is smaller than n_iter=1795. Running 216 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
